### **Presenting the Results**

Most of the implementations are in the script file, here we just assemble everything together and present the results for view. 
This is necessary since the script is very long. 
Run the cell below to import everthing from the pre-cooked script file. 



In [1]:
from sqp import *

TESTSTRING = "02002102000101000000102002110000021102201100000121101100102200102212002011112010100202020220021011110001020002102020211001102210020111001100000102100022100110201210022100020000101002210202100021000220220211202211110002221011010211000211202201021002102200012201101110222110002022012210202020020102100202211110202001122020000110020222220022110010020002102120002010010000211002021102102121210202221122000110202101020002020022200021000211020211022210200121022200010211002201101110220220110202110202210020212102102120002210002202112110210020001010002002000202102121222022121022201210211202020022100222101102112100221202021001010211020210102110202211200202000000000022102020000021111220012110201121010002002020000120200222022110202011002101002110010002120221100011000002100220222202021110222102200022001101011122021021111120021100010210222100222110202210102002221000021202020210200201101001120002211121011000212002000122022200121011120000210111011111020112221002002202"
# TESTSTRING = TESTSTRING[:40]
np.printoptions(suppress=True)


The following code uses what is written in the script file and call the SLSQP solver from scipy and solve the problem for a specific strings. 

In [2]:
def run_sqp(test_string, lmbd):
    global pbm
    pbm = ProblemModelingSQP(test_string, lmbd=lmbd)
    n = len(pbm.States)
    # Functional representation of Constraints =================================
    ineq_cons = {'type': 'ineq',
            'fun' : pbm.IneqCon,
            'jac' : pbm.IneqJac
        }
    eq_cons = {'type': 'eq',
            'fun' : pbm.EqnCon,
            'jac' : pbm.EqnJac
        }
    l = pbm.CRowsCount + len(pbm.TransFreqasVec)
    bounds = scipy.optimize.Bounds(
        np.zeros(l), np.full(l, np.inf)
    )
    # the objective function and gradient ======================================
    objfxn = pbm.ObjFxn
    objgrad = pbm.ObjGrad
    # initial Guess ============================================================
    C = pbm.ConstraintMatrixC
    x0 = pbm.TransitionMatrix.reshape(-1)
    # x0 = np.random.rand(n**2)
    x0 = np.hstack((x0, np.dot(C, x0))).copy()
    global res
    res = scipy.optimize.minimize(
            objfxn, x0, method='SLSQP', jac=objgrad,
            constraints=[eq_cons, ineq_cons], 
            options={'ftol': 1e-10,'disp': 2, "maxiter":5000},
            bounds=bounds, 
        )
    M = res.x[:n**2].reshape((n, n))
    return M, pbm.TransitionMatrix


The above function takes in a test string consist of characters representing the observed states from a markov chain. 
It will print out the optimize state transition matrix. 
It returns the optimize state transition matrix together with the original state transition matrix estimated via MLE. 
It setup the problem using the code written in the imported script, and then it solves using the `scipy.optimize.minimize` module using sequential quadratic programming. 
Each time the funtion is evaluated, it prints out the objective value of the function and a timestep accurated to miliseconds. 
In the cell below, we test it with some basic input. 

In [3]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.05)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)
print("Eqn con rhs: ")
print(pbm.EqnCon(res.x))
print("Ineq con rhs: ")
print(pbm.IneqCon(res.x))

/Users/hongdali/Desktop/repos/Operator-Lifting/src_python/sqp.py:130: RuntimeWarning: divide by zero encountered in log
  return x1*np.log(x2)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.546401915597745
            Iterations: 2160
            Function evaluations: 23572
            Gradient evaluations: 2157
The original MLE estimated transition matrix is: 


array([[0.40342298, 0.42298289, 0.17359413],
       [0.38566553, 0.27645051, 0.33788396],
       [0.5019305 , 0.15444015, 0.34362934]])

The New Transiiton Matrix after the SQP is: 


array([[4.65870248e-05, 1.49967225e-05, 9.99938416e-01],
       [2.41715412e-05, 9.99947419e-01, 2.84091937e-05],
       [1.23959371e-07, 9.92635645e-01, 7.36423146e-03]])

Eqn con rhs: 
[5.99520433e-15 5.10702591e-15 7.26751992e-13]
Ineq con rhs: 
[1.41036081e-10 9.99891831e-02 1.04755956e-10 9.99900833e-02
 1.34765529e-10 6.14794940e-09 9.92589163e-02 7.43826719e-04
 9.99923420e-02 9.17614670e-07 9.99932424e-02 1.34141574e-06
 3.15027367e-07 9.92620755e-02 7.49509193e-04 2.44105009e-10
 9.00399181e-07 1.61831919e-10 6.22591669e-09 6.80598769e-06
 4.73886069e-09 9.99923249e-02 4.23928472e-07 6.13858888e-09
 9.92611579e-02 7.77192277e-04 1.14705224e-10 6.10266612e-09
 2.17607485e-05 4.59035820e-09 6.16896876e-09 9.92607343e-02
 7.46297093e-04 9.92635688e-02 7.46363175e-04 1.50516083e-08
 3.15917127e-06 1.57668272e-10 2.24165312e-06 8.83944699e-11
 1.81791787e-06 4.65245449e-06 1.05678520e-08 1.20622756e-05
 7.95026961e-11 1.32797661e-10 1.49151587e-10 1.68616675e-10
 1.80230368e-06 1.06912027e-08 1.45857193e-05 9.99914247e-02
 9.79470786e-11 9.99910009e-02 9.99938355e-02 7.37083155e-04
 9.92574232e-02 1.53085123e-10 1.63217491e-10 2.41089677e-06
 1.061524

We now decrease the value of lamabda, the reguarlizatin term and see how it affects the results of the transition matrix. 

In [4]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.03)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 12.451073735946569
            Iterations: 732
            Function evaluations: 7603
            Gradient evaluations: 728
The original MLE estimated transition matrix is: 


array([[0.40342298, 0.42298289, 0.17359413],
       [0.38566553, 0.27645051, 0.33788396],
       [0.5019305 , 0.15444015, 0.34362934]])

The New Transiiton Matrix after the SQP is: 


array([[1.48504462e-08, 1.84798052e-07, 9.99999800e-01],
       [6.37301658e-06, 9.97206353e-01, 2.78727430e-03],
       [2.25450235e-08, 9.99999955e-01, 2.26052620e-08]])

In [5]:
res

 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 12.451073735946569
       x: [ 1.485e-08  1.848e-07 ...  2.766e-09  3.000e-02]
     nit: 732
     jac: [-8.649e-08 -1.027e-06 ...  1.000e+00  1.000e+00]
    nfev: 7603
    njev: 728